In [ ]:
import anthropic
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.messages.batch_create_params import Request

import json
import math
import pandas as pd

import functions.prompts as prompts

client = anthropic.Anthropic()

In [ ]:
import numpy as np

df = pd.read_csv("dump/csv/papers.csv")
df['rank'] = df['id'].apply(lambda x: x.split("_")[0])
df = df.loc[df['rank'].isin(["1", "25", "50", "75", "100"])].reset_index(drop=True)

len(df)

# random -> only real paper / is not in 1/25/50/75/100 -> sample n=10
# rand = df[:-100].loc[~df['rank'].isin(["1", "25", "50", "75", "100"])].sample(n=10, random_state=42)

# sample = pd.concat([ranks, rand]).reset_index(drop=True)
# sample

In [ ]:
def req(id, text, top5=True):
    return Request(
        custom_id=id,
        params=MessageCreateParamsNonStreaming(
            model="claude-3-5-haiku-20241022",
            max_tokens=1024,
            system=f"{prompts.top5() if top5 else prompts.analysis()}\nPlease respond in valid JSON format that matches this schema: {str(prompts.Top5Model.model_json_schema() if top5 else prompts.AnalysisModel.model_json_schema())}. **IMPORTANT**: ONLY RESPOND WITH A JSON OBJECT CONTAINING SCORES ACCORDING TO THE ABOVE SCHEMA. THE RESPONSE MUST END WITH A CURLY BRACKET. DO NOT ADD ANALYSIS OR EXPLANATION.",
            messages=[{
                "role": "user",
                "content": text
            }, {
                "role": "assistant",
                "content": "{"
            }]
        )
    )
    
def batch(text, id):
    return [    *[ req(f"{id}Z{i}Qtop5", text) for i in range(3) ],
                *[ req(f"{id}Z{i}Qanalysis", text, top5=False) for i in range(3) ] ]

In [ ]:
full_req = {}
divider = 1
for i in range(divider):
    full_req[f'batch-{i}'] = []

def partial(paper, no):
    l = len(paper.split())
    # if(no == 0.01):
        # print("asdf", math.ceil((l * no) / 100))
    return " ".join(paper.split()[:math.ceil((l * no) / 100)])

for index, row in df.iterrows():
    file_index = math.floor(index / (len(df) / divider))
    with open(f'output/{row["id"]}.txt', 'r') as f:
        text = f.read()
        paper = f"PAPER TITLE: {row['name']}\n\nPAPER TEXT: {text}"
        
        for no in [0.1, 1, 10, 50, 100]:
            full_req[f"batch-{file_index}"] += batch(partial(paper, no), row["id"]+"P"+str(no).split(".")[0])

In [ ]:
for i in range(divider):
    print(len(full_req[f'batch-{i}']))

In [ ]:
full_req[f'batch-{i}'][10]

In [ ]:
# KINDA DANGEROUS
batches = []
for i in range(divider):
    print(f"Sending Batch {i}")
    message_batch = client.messages.batches.create(
        requests=full_req[f'batch-{i}'])
    print(f"{i} {message_batch.id}")
    batches.append(message_batch)

print(batches)

In [ ]:
import time

id = ""
def wait(id):
    results = client.messages.batches.retrieve(id).processing_status
    while results == "in_progress":
        stat = client.messages.batches.retrieve(id)
        print(stat.request_counts)
        results = stat.processing_status
        time.sleep(5)

wait(id)

In [ ]:
batches = []
with open("dump/anthropic-batch/anthropic-batch-partial.txt", 'r') as f:
    batches = f.read()
    batches = batches.split("\n")

In [ ]:
def parse_r(r):
    id = r.custom_id
    validateModel = prompts.Top5Model if "top5" in id else prompts.AnalysisModel
    try:
        text = r.result.message.content[0].text
        text = "{" + text.split("}")[0] + "}"
        return {
            "id": id,
            "scores": validateModel.model_validate(json.loads(text)).model_dump()
        }
    except Exception as e:
        print(f"Error {e} - {"{" + r.result.message.content[0].text}")
        return {
            "id": id,
            "scores": None
        }

for b in batches:
    results = client.messages.batches.results(b)
    for r in results:
        if(r and r.result.type == 'succeeded'):
            try:
                with open('dump/eval-output/anthropic-result-partial.jsonl', 'a') as f:
                    f.write(json.dumps(parse_r(r)) + "\n")
            except Exception as e:
                print("ERROR! - " + str(e))

In [ ]:
import pandas as pd

df = pd.read_csv("dump/csv/segments.csv")

f = open('dump/eval-output/anthropic-result-partial.jsonl', 'r')
file_response = f.read()
f.close()

for line in file_response.split("\n")[:-1]:
    l = json.loads(line)
    id, s = l['id'].split("Z")[0].split("P")
    no, typ = l['id'].split("Z")[1].split("Q")
    
    idx = df.index[df['id'] == id].tolist()[0]
    
    # content = l['response']['body']['choices'][0]['message']['content']
    
    metrics = ['score'] if typ == "top5" else ['originality', 'rigor', 'scope', 'impact', 'written_by_ai']
    validateModel = prompts.Top5Model if typ == "top5" else prompts.AnalysisModel

    for i, metric in enumerate(metrics):
        column_name = f"anthropic-{s}-{metric}-{int(no)+1}"
        
        if column_name not in df.columns:
            df[column_name] = None

        o = validateModel.model_validate(l['scores'])
        df.loc[idx, column_name] = o.__dict__[metric]

In [ ]:
df['rank'] = df['id'].apply(lambda x: x.split("_")[0])
df.head()

In [ ]:
df.to_csv("dump/csv/result_partial.csv", index=False)